<a href="https://colab.research.google.com/github/DiMorten/osss-mcr/blob/coords7/train_and_evaluate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Dependencies

In [1]:
!pip install icecream
#%tensorflow_version 1.x
import os
!pip install kora
from kora import drive
import time
!pip install colorama
!pip install unrar

ds_path='/content/drive/My Drive/PhD/datasets/lm_data/'

In [2]:

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
git_clone = True

if git_clone == True:
  os.chdir('/content')
  %rm -rf osss-mcr
  !git clone --branch coords7 https://github.com/DiMorten/osss-mcr.git

Cloning into 'osss-mcr'...
remote: Enumerating objects: 2539, done.
remote: Counting objects: 100% (338/338), done.
remote: Compressing objects: 100% (251/251), done.
remote: Total 2539 (delta 230), reused 153 (delta 62), pack-reused 2201
Receiving objects: 100% (2539/2539), 37.67 MiB | 17.38 MiB/s, done.
Resolving deltas: 100% (1696/1696), done.


## Download images into proper folder

In [ ]:
# !cp -r /content/drive/MyDrive/PhD/datasets/lm_data /content/osss-mcr/dataset/

os.chdir('/content/osss-mcr/dataset/')
!gdown --id 1MPqRKFcT2GzWGQM7PO7vy8X-smf5kFAV

Downloading...
From: https://drive.google.com/uc?id=1MPqRKFcT2GzWGQM7PO7vy8X-smf5kFAV
To: /content/osss-mcr/dataset/lm_data.rar
3.07GB [00:21, 33.2MB/s]

In [ ]:
!unrar x -y "lm_data.rar" "/content/osss-mcr/dataset/"

In [ ]:
os.chdir('/content/osss-mcr/')
os.getcwd()
os.listdir()

In [ ]:
from colorama import init
init()
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPool2D, Flatten, Dropout, Conv2DTranspose
# from tensorflow.keras.callbacks import ModelCheckpoint , EarlyStopping
from tensorflow.keras.optimizers import Adam,Adagrad 
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
import tensorflow.keras as keras

import numpy as np
from sklearn.utils import shuffle
import cv2
import argparse
import tensorflow as tf

from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras import metrics
import sys
import glob

from sklearn.metrics import confusion_matrix,f1_score,accuracy_score,classification_report
# Local
from src.densnet import DenseNetFCN
from src.densnet_timedistributed import DenseNetFCNTimeDistributed

#from metrics import fmeasure,categorical_accuracy
import deb
from src.keras_weighted_categorical_crossentropy import weighted_categorical_crossentropy, sparse_accuracy_ignoring_last_label, weighted_categorical_crossentropy_ignoring_last_label, categorical_focal_ignoring_last_label, weighted_categorical_focal_ignoring_last_label
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import ConvLSTM2D, UpSampling2D, multiply
from tensorflow.keras.regularizers import l1,l2
import time
import pickle
#from tensorflow.keras_self_attention import SeqSelfAttention
import pdb
import pathlib
from pathlib import Path, PureWindowsPath
from tensorflow.keras.layers import Conv3DTranspose, Conv3D

from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
from collections import Counter


from src.generator import DataGenerator, DataGeneratorWithCoords, DataGeneratorWithCoordsRandom

import matplotlib.pyplot as plt
sys.path.append('../../../dataset/dataset/patches_extract_script/')
from src.dataSource import DataSource, SARSource, Dataset, LEM, LEM2, CampoVerde
from src.model_input_mode import MIMFixed, MIMVarLabel, MIMVarSeqLabel, MIMVarLabel_PaddedSeq, MIMFixedLabelAllLabels, MIMFixed_PaddedSeq
from parameters.params_train import ParamsTrain
from parameters.params_mosaic import ParamsReconstruct

from icecream import ic
from src.monitor import Monitor, MonitorNPY, MonitorGenerator, MonitorNPYAndGenerator
import natsort
from src.model import NetModel, ModelFit, ModelLoadGeneratorWithCoords
from src.dataset import Dataset, DatasetWithCoords

from src.patch_extractor import PatchExtractor

from src.mosaic import seq_add_padding, add_padding, Mosaic, MosaicHighRAM, MosaicHighRAMPostProcessing
from src.postprocessing import PostProcessingMosaic

from src.metrics import Metrics, MetricsTranslated

ic.configureOutput(includeContext=True)
np.random.seed(2021)
tf.random.set_seed(2021)
tf.compat.v1.disable_eager_execution()
tf.compat.v1.experimental.output_all_intermediates(True)

from train_and_evaluate import TrainTest

In [ ]:
!nvidia-smi

## Set parameters

Parameters can be modified in /content/osss-mcr/networks/convlstm_networks/train_src/parameters/params_train.py

In [ ]:
paramsTrainCustom = {
	'getFullIms': True,
	'coordsExtract': True,
	'train': True,
	'openSetMethod': None, # Options: None, OpenPCS, OpenPCS++
#		'openSetLoadModel': True,
	'selectMainClasses': True,
	'dataset': 'lm', # lm: L Eduardo Magalhaes.
	'seq_date': 'mar',	# jun, mar	
    'dataSource': SARSource()
}

paramsTrain = ParamsTrain('parameters/', **paramsTrainCustom)

paramsTrain.dataSource = SARSource()

trainTest = TrainTest(paramsTrain)



In [ ]:
trainTest.main()